In [1]:
from IPython.display import Markdown, display
display(Markdown(open("./SM_header.md", "r").read()))

Copyright © 2025 Université Paris Cité

Author: [Guillaume Rousseau](https://www.linkedin.com/in/grouss/), Department of Physics, Paris, France (email: guillaume.rousseau@u-paris.fr)

This archive contains the supplemental materials and replication package associated with the preprint, "*Temporal and topological partitioning in real-world growing networks for scale-free properties study*", available on [arXiv](https://arxiv.org/abs/2501.10145) and [ssrn](http://ssrn.com/abstract=5191689).

**The latest version of the preprint (timestamped arXiv:2501.10145v2) is downloadable here https://arxiv.org/pdf/2501.10145v2**

The current version of the Python scripts and associated resources is available on the [author's GitHub page](https://github.com/grouss/growing-network-study).

This work is currently licensed under the [Creative Commons CC BY-NC-SA 4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0).

To give appropriate credit and cite this work ([BibTeX entry](./rousseau2025temporal)):
Rousseau, G. (2025). *Temporal and topological partitioning in real-world growing networks for scale-free properties study* [Preprint]. arXiv:2501.10145. https://arxiv.org/abs/2501.10145; also available on SSRN: http://ssrn.com/abstract=5191689

 
# A) Replication Packages

[Open the Replication Package notebook related to the datasets.](./Replication_Package_Datasets.ipynb)

[Open the Replication Package notebook related to the figures.](./Replication_Package_Figures.ipynb)

# B) QuickStart Guide

[Open the QuickStart Guide notebook](./SM00_QuickStart.ipynb)

# C) Table of Contents

- 1. [Function Definitions](./SM01_Functions.ipynb)
- 2. [Dataset Import](./SM02_DatasetImport.ipynb)
- 3. [Building the Transposed Graph](./SM03_BuildingTransposedGraph.ipynb)
- 4. [Temporal Information Quality and Summary Statistics](./SM04_TemporalInformationMainStats.ipynb)
- 5. [Growth Relationship Between Nodes and Edges](./SM05_GrowingRules.ipynb)
- 6. [Topological Partitioning($RV$ Nodes)](./SM06_TopologicalPartitioning.ipynb)
- 7. [In-Degree and Out-Degree Distributions Over Time](./SM07_DegreeDistributionOverTime.ipynb)
- 8. [Distribution Tail Analysis](./SM08_DistributionTailAnalysis.ipynb)
- 9. [Temporal Partitioning](./SM09_TemporalPartitioning.ipynb)
- 10. [Derived $O-(RV/RL)-O$ Graph Construction](./SM10_DerivedGrowingNetwork.ipynb)
- 11. [Building the $TSL$ Partitioning](./SM11_TSLPartitioning.ipynb)
- 12. [Barabási–Albert Model Use Case](./SM12_BarabasiAlbertUseCase.ipynb)


**NB :** As of 2025/05/16, the QuickStart guide, the replication packages, and SM01 to SM12 are available. The Python scripts are also provided under `local_utils` directory, but they are not in their final form and should be considered an alpha release. The graph datasets used in the study are available in a distinct Zenodo Deposit 10.5281/zenodo.15260640 ($\sim50$ Go), including the main dataset $O/RV/RL-O/RV/RL$ (2+ billions of nodes, $\sim4$ billions of edges), and two derived $O-(RV/RL)-O$ graphs ($\sim150$ millions nodes and edges). 

More release notes are available in the [dedicated notebook](./SM_ReleaseNote.ipynb).

In [2]:
%load_ext autoreload
%autoreload 2

import importlib,sys,local_utils
from local_utils import *

print("___ Import data from graphpath=",config.graphpath)
print("___ Export data to exportpath=",config.exportpath)   

DisplayCopyrightInfo()


___ Import data from graphpath= ./ImportData/
___ Export data to exportpath= ./ExportData/
--------------------------------------------------------------------------------
Copyright 2025 Université Paris Cité, France 
Author: Guillaume Rousseau, Physics Department, Paris, France 

(https://www.linkedin.com/in/grouss/)

This archive contains the supplemental materials and replication package associated with the preprint available on :
- arXiv (https://arxiv.org/abs/2501.10145)
- SSRN  (http://ssrn.com/abstract=5191689

Current version of python scripts and associated ressources are available on author's github page
(https://github.com/grouss/growing-network-study)

This work is currently licensed under CC BY-NC-SA 4.0
(https://creativecommons.org/licenses/by-nc-sa/4.0)
--------------------------------------------------------------------------------



# 3. Building transposed graph 

The construction of the transposed graph from the numpy arrays `nodes` and `edges` can be done using methods from the `numpy` library.


In [11]:
# Here we present a step by step building of the transpose of the main 
# graph cleaning edges and nodes to minimize inmemory footprint
# 
try:
    del nodes
except:
    pass
try:
    del edges
except:
    pass
    
DEBUG=True
BENCHMARK=True

# nodes (2B+) and edges (~4B) file of the main graph
filename_nodes=graphpath+"nodes_20240310.pkl"
filename_edges=graphpath+"edges_20240310.pkl"
# temporay file
filename_argsort=graphpath+"edges_argsort_20240310.pkl"

Nnodes=2224378840
Nedges=3841679043

## a) Step by step method

In [12]:
print("Edges Load start")
ti=time.time()
edges=pickle.load(open(filename_edges,"rb"))
tf=time.time()
print("Edges Load elapse(s)",np.round(tf-ti),"(s)")

print("Edges Argsort Start")
ti=time.time()
try:
    argsortedges=pickle.load(open(filename_argsort,"rb"))
    print("We find it and use",filename_argsort)
except:
    argsortedges=ArgSort(edges) # use default quicksort
    if DEBUG:
        # if one want to save it (but it's a temporay array)
        pickle.dump(argsortedges,open(filename_argsort,"wb"))
tf=time.time()
print("Edges Argsort elapse(s)",np.round(tf-ti),"(s)")

ti=time.time()
nodes_transpose=GetNodesTranspose(edges,Nnodes)
tf=time.time()
print("Del edges array, no longer needed to build transposed derived graph")
print("Nodes Transpose elapse(s)",np.round(tf-ti),"(s)")
if not DEBUG:
    print("Del edges, no longer needed to build transposed derived graph")
    del edges # no longer needed to build transposed derived graph

print("Nodes transpose Dump Start")
ti=time.time()
if not BENCHMARK:
    # False for benchmark prupose
    pickle.dump(nodes_transpose,open(filename_nodes.replace("nodes","nodes_transpose"),"wb"))
else:
    print("Not saved for benchmark purpose")
tf=time.time()
print("Nodes transpose Dump elapse(s)",np.round(tf-ti),"(s)")
print("nodes_transpose",nodes_transpose)
if not DEBUG:
    print("Del nodes_transpose, no longer needed to build transposed derived graph")
    del nodes_transpose

print("Read nodes array Start")
ti=time.time()
nodes=pickle.load(open(filename_nodes,"rb"))
tf=time.time()
print("Read nodes array elapse(s)",np.round(tf-ti),"(s)")

edges_transpose=GetEdgesTranspose(nodes,argsortedges,Nedges)

print("Del argsortedges,nodes, no longer needed to build transposed derived graph")
del argsortedges
del nodes

print("edges_transpose",edges_transpose)
print("Nodes transpose Dump Start")
ti=time.time()
if not BENCHMARK:
    pickle.dump(edges_transpose,open(filename_edges.replace("edges","edges_transpose"),"wb"))   
else:
    print("Not saved for benchmark purpose")
tf=time.time()
print("Edges transpose Dump elapse(s)",np.round(tf-ti),"(s)")

if not DEBUG:
    print("Del edges_transpose, no longer needed to build transposed derived graph")
    del edges_transpose

Edges Load start
Edges Load elapse(s) 1.0 (s)
Edges Argsort Start
We find it and use ./ImportData/edges_argsort_20240310.pkl
Edges Argsort elapse(s) 4.0 (s)
Nodes Transpose Start
Nodes transpose Start
Nodes transpose elapse(s) 37.0 (s)
Del edges array, no longer needed to build transposed derived graph
Nodes Transpose elapse(s) 37.0 (s)
Nodes transpose Dump Start
Not saved for benchmark purpose
Nodes transpose Dump elapse(s) 0.0 (s)
nodes_transpose [         0          0          0 ... 3841679041 3841679042 3841679043]
Read nodes array Start
Read nodes array elapse(s) 4.0 (s)
Edges transpose Start
Edges transpose Step 1/2 elapse(s) 20.0 (s)
Edges transpose Step 2/2 elapse(s) 27.0 (s)
Del argsortedges,nodes, no longer needed to build transposed derived graph
edges_transpose [  46399055   10841856  115097518 ...  116034996 1914050848  166003460]
Nodes transpose Dump Start
Not saved for benchmark purpose
Edges transpose Dump elapse(s) 0.0 (s)


In [ ]:
Edges Load start
Edges Load elapse(s) 1.0 (s)
Edges Argsort Start
Edges Argsort elapse(s) 677.0 (s)
Nodes transpose Start
Nodes transpose elapse(s) 37.0 (s)
Del edges array, no longer needed to build transposed derived graph
Nodes Transpose elapse(s) 37.0 (s)
Nodes transpose Dump Start
Not saved for benchmark purpose
Nodes transpose Dump elapse(s) 0.0 (s)
nodes_transpose [         0          0          0 ... 3841679041 3841679042 3841679043]
Read nodes array Start
Read nodes array elapse(s) 4.0 (s)
Edges transpose Start
Edges transpose Step 1/2 elapse(s) 20.0 (s)
Edges transpose Step 2/2 elapse(s) 27.0 (s)
Del argsortedges,nodes, no longer needed to build transposed derived graph
edges_transpose [  46399055   10841856  115097518 ...  116034996 1914050848  166003460]
Nodes transpose Dump Start
Not saved for benchmark purpose
Edges transpose Dump elapse(s) 0.0 (s)

**Comment**: It takes less than 15 minutes to build the transpose of the main graph on a Mac Studio Ultra (Apple M2 chip). It is worth noting that two-thirds of the time is spent ordering the source/target edge arrays. This explains why it is useful to benchmark and select the most efficient sorting algorithm.

## b) Sort algorithm benchmark

In [8]:
# Sorting performance often depends on the "worst" or "best" case scenarios in terms of algorithmic complexity.
# See: https://numpy.org/doc/stable/reference/generated/numpy.sort.html#numpy.sort
# A benchmark of NumPy's argsort can be performed to evaluate its behavior on this specific graph.
if BENCHMARK:
    for Nslice in [1000,100,50]:
        arraybench=pickle.load(open(filename_edges,"rb"))[::Nslice]
        print(" edges ::"+str(Nslice))
        for kind in ["quicksort", "mergesort", "heapsort"]:
            ti=time.time()
            tmparray=ArgSort(arraybench,kind=kind)
            tf=time.time()
            print("ArgSort transpose bench elapse(s) kind=",kind,":",np.round(tf-ti,2),"(s)")    
        del arraybench


 edges ::1000
ArgSort transpose bench elapse(s) kind= quicksort : 0.29 (s)
ArgSort transpose bench elapse(s) kind= mergesort : 0.41 (s)
ArgSort transpose bench elapse(s) kind= heapsort : 1.03 (s)
 edges ::100
ArgSort transpose bench elapse(s) kind= quicksort : 4.07 (s)
ArgSort transpose bench elapse(s) kind= mergesort : 11.2 (s)
ArgSort transpose bench elapse(s) kind= heapsort : 35.87 (s)
 edges ::50
ArgSort transpose bench elapse(s) kind= quicksort : 8.81 (s)
ArgSort transpose bench elapse(s) kind= mergesort : 29.89 (s)
ArgSort transpose bench elapse(s) kind= heapsort : 89.62 (s)


## c) Stats of the transposed graph

In [ ]:
DEBUG=False
if DEBUG:
    print("Nodes and edges stats of the TRANSPOSED graph");print()
    nodes_transpose,edges_transpose,nodesad,d,Nnodes,Nedges=LoadAllArray(transpose=True)
    DisplayTypeStats(nodes_transpose,edges_transpose,d)    

In [ ]:
DEBUG=False
if DEBUG:
    print("Nodes and edges stats of the graph");print()
    nodes,edges,nodesad,d,Nnodes,Nedges=LoadAllArray()   
    del nodesad
    DisplayTypeStats(nodes,edges,d)    

## d) Integrity Check

Statistics of the graph and its transposed version show, for instance, that the number of `RV>O` edges in the transposed graph is the same as `O>RV` edges in the original graph — which is, of course, the expected behavior.

We can also perform random tests to check that edges exist in both the transposed and original graphs.


In [ ]:
if DEBUG:
    def CheckTransposeDirect(index,nodes,edges,nodes_transpose,edges_transpose,direction):
        nedges=nodes[index+1]-nodes[index]
        if np.sum(edges_transpose==index)!=nedges:
            print("ERROR 1",direction,index,nedges,np.sum(edges_transpose==index))
        directedges=edges[nodes[index]:nodes[index+1]]
        
        for index2 in directedges:
            transposeedges=edges_transpose[nodes_transpose[index2]:nodes_transpose[index2+1]]
            if index not in transposeedges:
                print("ERROR 2",direction,index,index2)
                
    # randomly check that edges exists in both transposed and direct graph.
    Ncheck=10 # for each node type + boundary
    for nodetype in ["O","RL","RV"]:
        print("Check NodeType ",nodetype)
        indexList=[]
        for boundary in ["Min","Max"]:
            indexList.append(d[nodetype+"index"+boundary])
        indexList+=list(np.random.randint(d[nodetype+"indexMin"],d[nodetype+"indexMax"]+1,Ncheck))
        for i,index in enumerate(indexList):
            print(i,end=" ")
            CheckTransposeDirect(index,nodes,edges,nodes_transpose,edges_transpose,"downstream")
            CheckTransposeDirect(index,nodes_transpose,edges_transpose,nodes,edges,"upstream")
        print()
            
        
#Check NodeType  O
#0 1 2 3 4 5 6 7 8 9 10 11 
#Check NodeType  RL
#0 1 2 3 4 5 6 7 8 9 10 11 
#Check NodeType  RV
#0 1 2 3 4 5 6 7 8 9 10 11   

